In [1]:
import os
import json
import pandas as pd
import psycopg2
from psycopg2 import OperationalError
from psycopg2 import ProgrammingError


In [2]:
# read user file
with open('package/account/user.json', 'r') as read_file:
    user = json.load(read_file)

In [3]:
# make discord name
discord_username_full = user['username'] + '#' + str(user['discriminator'])

In [4]:
# function to connect to postgresql db
def create_connection(db_name, db_user, db_password, db_host, db_port):
    try:
        connection = psycopg2.connect(
            database=db_name,
            user=db_user,
            password=db_password,
            host=db_host,
            port=db_port,
        )
        print('Connection to PostgreSQL DB successful')
    except OperationalError as e:
        connection = None
        print(f"The error '{e}' occurred")
        print('Error Type:', type(e))

    return connection

In [5]:
def read_secrets() -> dict:
    filename = os.path.join('secrets.json')
    try:
        with open(filename, mode='r') as f:
            return json.loads(f.read())
    except FileNotFoundError:
        return {}

In [6]:
secrets = read_secrets()

In [7]:
# set up database connection
con = create_connection(secrets['db_name'], secrets['db_user'],
                        secrets['db_password'], secrets['db_host'], secrets['db_port'])



Connection to PostgreSQL DB successful


In [8]:
cur = con.cursor()

In [9]:
# function to select data, returns a dataframe
def select(query):
    data_df = None
    try:
        cur.execute(query)
        data_df = pd.DataFrame(cur.fetchall())
    except ProgrammingError as e:
        con.rollback()
        print(f"The error '{e}' occurred")
        print('Error Type:', type(e))

    return data_df

In [10]:
select_query = """
SELECT *
FROM discord_data.dim_channel_type
"""

df = select(select_query)

In [11]:
df


,0,1
0,0,Server Channel
1,1,DM
2,2,Server Voice Text Channel
3,3,Group DM
4,4,Server Category
5,5,Server Annoucements
6,10,Server Announcment Thread
7,11,Server Channel Public Thread
8,12,Server Channel Private Thread
9,13,Stage Voice Channel
